## MongoDB
MongoDB一種強大，靈活、且易於擴展的文件導向式(document-oriented)資料庫，與傳統的關聯式導向資料庫相比，它不再有row的概念，取而代之的是document的概念

面向檔案的儲存:以JSON格式的檔案來保存數據

## 載入MongoDB

In [1]:
import requests
from pymongo import MongoClient

##  建立連結
命名其中一個database

In [2]:
client=MongoClient()
db=client['nobel']

## insert_many insert_one 
insert_many: 批量插入資料集

insert_one: 插入一個row,包含column名稱

In [3]:
for collection_name in ['prizes', 'laureates']:
    singular = collection_name[:-1]
    response = requests.get("http://api.nobelprize.org/v1/{}.json".format(singular))

    # Returns the json-encoded content of a response
"""
加入document進入database裡
"""
    documents = response.json()[collection_name]
    db[collection_name].insert_many(documents)

## count_documents 
計算該欄位的資訊

In [143]:
assert client.nobel == db
assert db.prizes == db['prizes']

# Count documents
n_prizes=db.prizes.count_documents({})
n_laureates=db.laureates.count_documents({})

print(f'有 {n_prizes} 個prizes資料')
print(f'有 {n_laureates} 個laureates資料')

有 1180 個prizes資料
有 1868 個laureates資料


In [150]:
# $出生在1700年之前
db.laureates.count_documents({"born": {"$lt": "1700"}})

76

In [151]:
# Composing filters 找到自己指定的內容
# match的功能
criteria = {'diedCountry': 'USA', 'bornCountry': 'Germany'}
db.laureates.count_documents(criteria)

10

In [168]:
db.laureates.count_documents({"prizes.share": "1","prizes.year": {"$gte": "1945"}})

414

## find_one
找出其中一筆資料

In [144]:
# Find one document to inspect
# 找出prizes裡面的其中一筆資料
doc = db.prizes.find_one({})

doc

{'_id': ObjectId('5c7a5900db0b51193a88b28c'),
 'year': '2018',
 'category': 'physics',
 'overallMotivation': '“for groundbreaking inventions in the field of laser physics”',
 'laureates': [{'id': '960',
   'firstname': 'Arthur',
   'surname': 'Ashkin',
   'motivation': '"for the optical tweezers and their application to biological systems"',
   'share': '2'},
  {'id': '961',
   'firstname': 'Gérard',
   'surname': 'Mourou',
   'motivation': '"for their method of generating high-intensity, ultra-short optical pulses"',
   'share': '4'},
  {'id': '962',
   'firstname': 'Donna',
   'surname': 'Strickland',
   'motivation': '"for their method of generating high-intensity, ultra-short optical pulses"',
   'share': '4'}]}

## list_database_names list_collection_names

list_database_names : 列出database名稱

list_collection_names : 列出database裡不同資料集的名稱

In [7]:
# Save a list of names of the databases managed by client
db_names=client.list_database_names()
print(db_names)

# Save a list of names of the collections managed by the "nobel" database
nobel_coll_names=client.nobel.list_collection_names()
print(nobel_coll_names)

['admin', 'config', 'local', 'nobel']
['laureates', 'prizes']


In [152]:
## Compute the total number of laureate prizes
print(sum(len(doc['prizes']) for doc in db.laureates.find()))

1882


## Operator 

### $in 

#$in : 可以指定不同類型的條件和值,如正在將使用者的ID號遷移成使用者名的過程中,要做到兩者兼顧的查詢

In [120]:
count={'bornCountry':{'$in':['USA','Canada','Mexico']}}
print(db.laureates.count_documents(count))

582


### $ne

$ne : not equal

In [12]:
#$ne (not equal)
count={'bornCountry':{'$ne':'USA'},'diedCountry':'USA'}
print(db.laureates.count_documents(count))

138


In [13]:
criteria = {'bornCountry': 'Austria',
            'prizes.affiliations.country': {"$ne": 'Austria'}}
db.laureates.count_documents(criteria)

20

### $exist

$exists用來判斷一個元素是否存在

如果存在元素a,就返回  存在表示使用True

如果不存在元素a，就返回 不存在表示False

In [167]:
db.laureates.find_one({'id':'482'})

{'_id': ObjectId('5c7a5903db0b51193a88b6b5'),
 'id': '482',
 'firstname': 'Comité international de la Croix Rouge (International Committee of the Red Cross)',
 'born': '0000-00-00',
 'died': '0000-00-00',
 'gender': 'org',
 'prizes': [{'year': '1917',
   'category': 'peace',
   'share': '1',
   'affiliations': [[]]},
  {'year': '1944', 'category': 'peace', 'share': '1', 'affiliations': [[]]},
  {'year': '1963', 'category': 'peace', 'share': '2', 'affiliations': [[]]}]}

In [154]:
criteria = {'born': {'$exists': False}}
assert db.laureates.count_documents(criteria) == 0

In [15]:
criteria = {"prizes.2": {"$exists": True}}
print(db.laureates.find_one(criteria))

{'_id': ObjectId('5c7a5903db0b51193a88b6b5'), 'id': '482', 'firstname': 'Comité international de la Croix Rouge (International Committee of the Red Cross)', 'born': '0000-00-00', 'died': '0000-00-00', 'gender': 'org', 'prizes': [{'year': '1917', 'category': 'peace', 'share': '1', 'affiliations': [[]]}, {'year': '1944', 'category': 'peace', 'share': '1', 'affiliations': [[]]}, {'year': '1963', 'category': 'peace', 'share': '2', 'affiliations': [[]]}]}


## .distinct
返回unique的值

In [16]:
countries = set(db.laureates.distinct('diedCountry')) - set(db.laureates.distinct('bornCountry'))
print(countries)

{'USSR', 'Barbados', 'Northern Rhodesia (now Zambia)', 'Philippines', 'Jamaica', 'East Germany', 'Czechoslovakia', 'Yugoslavia (now Serbia)', 'Israel', 'Tunisia', 'Greece', 'Gabon', 'Puerto Rico'}


### 有條件下的distinct 

In [17]:
db.laureates.distinct('prizes.affiliations.country',{'bornCountry':'USA'})

['USA', 'Denmark', 'Australia', 'United Kingdom']

##  $elemMatch
查询Array Field中的元素是否满足一系列的查詢條件。

In [169]:
db.laureates.count_documents({"prizes": {"$elemMatch": {"share": "1", "year":  {"$gte": "1945"}}}})

412

In [20]:
db.laureates.count_documents({"prizes": {"$elemMatch": {'share':'1',"category": "physics", "year":  {"$gte": "1945"}}}})

36

## ＄lt $gte 

$$$lt專指在某年之前$$

$$$gte指在某年之後(包括該年)$$

In [21]:
before = {
    "gender": "org",
    "prizes.year": {"$lt": "1945"},
    }

# Save a filter for organization laureates with prizes won in or after 1945
in_or_after = {
    "gender": "org",
    "prizes.year": {"$gte": "1945"},
    }

n_before = db.laureates.count_documents(before)
n_in_or_after = db.laureates.count_documents(in_or_after)
ratio = n_in_or_after / (n_in_or_after + n_before)
print(ratio)

0.84


## Regex

In [170]:
from bson.regex import Regex

criteria = {"bornCountry": Regex('Germany')}
print(set(db.laureates.distinct("bornCountry", filter=criteria)))

{'Hesse-Kassel (now Germany)', 'Germany', 'East Friesland (now Germany)', 'Prussia (now Germany)', 'Germany (now Russia)', 'Bavaria (now Germany)', 'Schleswig (now Germany)', 'Germany (now France)', 'Germany (now Poland)', 'West Germany (now Germany)', 'Mecklenburg (now Germany)', 'W&uuml;rttemberg (now Germany)'}


###  ^ $

^S代表開頭為S的字母

S$代表結束為S的字母

In [25]:
from bson.regex import Regex

db.laureates.count_documents({"firstname": Regex('^G'), "surname": Regex('^S')})

18

In [26]:
criteria = {"bornCountry": Regex("^" + 'Germany \\(' + "now")}
print(set(db.laureates.distinct("bornCountry", criteria)))

{'Germany (now Russia)', 'Germany (now France)', 'Germany (now Poland)'}


In [27]:
criteria={'bornCountry':Regex('now'+' Germany\)'+'$')}
print(set(db.laureates.distinct('bornCountry',criteria)))

{'Hesse-Kassel (now Germany)', 'East Friesland (now Germany)', 'Prussia (now Germany)', 'Bavaria (now Germany)', 'Schleswig (now Germany)', 'West Germany (now Germany)', 'Mecklenburg (now Germany)', 'W&uuml;rttemberg (now Germany)'}


## projections
projection用于指定返回哪些字段

顯示的是一個文檔的所有字段。要想限制，可以利用 0 或 1 來設置字段列表。 1 用於顯示字段，0 用於隱藏字段。

### {}
可以用{}大括號找出projections {}意思為找出全部裡面的資料

In [172]:
list(
    db.laureates.find({}, {                #filter
        "prizes.category": "physics", #projection
        "prizes.year": "1903"
    }))[:3]

[{'_id': ObjectId('5c7a5903db0b51193a88b4da'),
  'prizes': [{'year': '1901', 'category': 'physics'}]},
 {'_id': ObjectId('5c7a5903db0b51193a88b4db'),
  'prizes': [{'year': '1902', 'category': 'physics'}]},
 {'_id': ObjectId('5c7a5903db0b51193a88b4dc'),
  'prizes': [{'year': '1902', 'category': 'physics'}]}]

In [173]:
from fractions import Fraction

docs=list(db.prizes.find({},['laureates.share']))

check=all(1== sum(Fraction(1,int(laureate['share']))
        for laureate in doc['laureates'])
                 for doc in docs)

print(check)

True


### []
用[]中括號找出projections

In [35]:
name = [
    '  '.join([doc['firstname'], doc['surname']])
    for doc in (db.laureates.find({
        "firstname": {
            "$regex": "^G"
        },
        "surname": {
            "$regex": "^S"
        }
    }, ["firstname", "surname"]))
]

print(name)

['Glenn Theodore  Seaborg', 'George D.  Snell', 'Gustav  Stresemann', 'George Bernard  Shaw', 'Giorgos  Seferis', 'George J.  Stigler', 'George F.  Smoot', 'George E.  Smith', 'George P.  Smith', 'Glenn Theodore  Seaborg', 'George D.  Snell', 'Gustav  Stresemann', 'George Bernard  Shaw', 'Giorgos  Seferis', 'George J.  Stigler', 'George F.  Smoot', 'George E.  Smith', 'George P.  Smith']


## sort 

1 for ascending and -1 for descending.

sort : 先call誰 就開始sort誰

In [66]:
docs = list(db.laureates.find(
    {"born": {"$gte": "1900"}, "prizes.year": {"$gte": "1954"}},
    {"born": 1, "prizes.year": 1, "_id": 0},
    sort=[('prizes.year',1),('born',-1)]))
for doc in docs[:5]:
    print(doc)

{'born': '1916-08-25', 'prizes': [{'year': '1954'}]}
{'born': '1916-08-25', 'prizes': [{'year': '1954'}]}
{'born': '1915-06-15', 'prizes': [{'year': '1954'}]}
{'born': '1915-06-15', 'prizes': [{'year': '1954'}]}
{'born': '1901-02-28', 'prizes': [{'year': '1954'}, {'year': '1962'}]}


### 用itemgeter
operator.itemgetter取出想排序的元素

In [181]:
from operator import itemgetter

# Sort by ascending year
sort_spec = [("year", 1)]

# Construct a cursor over physics prizes
cursor = db.prizes.find({"category": "physics"},
                        ["year", "laureates.firstname", "laureates.surname"],
                        sort=sort_spec)
docs = list(cursor)
cursor.rewind()


# Define a function names() to return a list of formatted names
def names(doc):
    formatted_names = [
        "{firstname} {surname}".format(**laureate)
        for laureate in sorted(doc["laureates"], key=itemgetter("surname"))
    ]
    return formatted_names


#每年進行合併
lines = [
    "{year}: {names}".format(year=doc["year"], names=" and ".join(names(doc)))
    for doc in cursor
]
for line in lines:
    print(line)

1901: Wilhelm Conrad Röntgen
1901: Wilhelm Conrad Röntgen
1902: Hendrik Antoon Lorentz and Pieter Zeeman
1902: Hendrik Antoon Lorentz and Pieter Zeeman
1903: Antoine Henri Becquerel and Pierre Curie and Marie Curie, née Sklodowska
1903: Antoine Henri Becquerel and Pierre Curie and Marie Curie, née Sklodowska
1904: Lord Rayleigh (John William Strutt)
1904: Lord Rayleigh (John William Strutt)
1905: Philipp Eduard Anton von Lenard
1905: Philipp Eduard Anton von Lenard
1906: Joseph John Thomson
1906: Joseph John Thomson
1907: Albert Abraham Michelson
1907: Albert Abraham Michelson
1908: Gabriel Lippmann
1908: Gabriel Lippmann
1909: Karl Ferdinand Braun and Guglielmo Marconi
1909: Karl Ferdinand Braun and Guglielmo Marconi
1910: Johannes Diderik van der Waals
1910: Johannes Diderik van der Waals
1911: Wilhelm Wien
1911: Wilhelm Wien
1912: Nils Gustaf Dalén
1912: Nils Gustaf Dalén
1913: Heike Kamerlingh Onnes
1913: Heike Kamerlingh Onnes
1914: Max von Laue
1914: Max von Laue
1915: Sir Willia

## gt It gte lte
$gt:大於

$lt:小於

$gte:大於或等於

$lte:小於或等於

In [77]:
db.prizes.distinct("category", {"laureates.share": {"$gt": "3"}})

['physics', 'chemistry', 'medicine']

## index
mongoDB可以建立index以利未來的searching，在以後searching時如果是searching有建立index的欄位，那麼mongoDB就不會進去資料欄位，而是調出己經建好B tree的index來查詢。


In [83]:
#針對model去做index
index_model = [("category", 1), ("year", -1)]
db.prizes.create_index(index_model)

# Collect the last single-laureate year for each category
report = ""
for category in sorted(db.prizes.distinct("category")):
    doc = db.prizes.find_one({"category": category, "laureates.share": "1"}, sort=[("year", -1)])
    report += "{category}: {year}\n".format(**doc)
print(report)

chemistry: 2011
economics: 2017
literature: 2017
medicine: 2016
peace: 2017
physics: 1992



In [86]:
from collections import Counter

# Ensure an index on country of birth
# 1代表pymongo.ASCENDING
db.laureates.create_index([("bornCountry", 1)])

# Collect a count of laureates for each country of birth
n_born_and_affiliated = {
    country: db.laureates.count_documents({"bornCountry": country, "prizes.affiliations.country": country})
    for country in db.laureates.distinct("bornCountry")
}

#分別計算不同的字串次數
five_most_common = Counter(n_born_and_affiliated).most_common(5)
print(five_most_common)

482
[('USA', 482), ('United Kingdom', 112), ('France', 52), ('Germany', 38), ('Japan', 34)]


## limit
如果call兩個limit,會回傳後面的limit

因為很像是d = {'limit': 3}; d.update({'limit': 5}); 

In [100]:
list(db.prizes.find({"category": "economics"},
                    {"year": 1, "_id": 0})
     .sort("year")
     .limit(3)
     .limit(5))

[{'year': '1969'},
 {'year': '1969'},
 {'year': '1970'},
 {'year': '1970'},
 {'year': '1971'}]

In [182]:
for doc in (db.prizes.find({"laureates.share": "3"}).limit(5)):
    print("{year} {category}".format(**doc))

2017 chemistry
2017 medicine
2016 chemistry
2015 chemistry
2014 physics


## skip 
跳過2個

In [190]:
for doc in (db.prizes.find({"laureates.share": "3"}).skip(2).limit(3)):
    print("{year} {category}".format(**doc))

2016 chemistry
2015 chemistry
2014 physics


## aggregate

In [109]:
# Translate cursor to aggregation pipeline
pipeline = [
    {'$match': {'gender': {'$ne': 'org'}}},
    {'$project': {'bornCountry': 1, 'prizes.affiliations.country': 1}},
    {'$limit': 3}
]

for doc in db.laureates.aggregate(pipeline):
    print("{bornCountry}: {prizes}".format(**doc))

Prussia (now Germany): [{'affiliations': [{'country': 'Germany'}]}]
the Netherlands: [{'affiliations': [{'country': 'the Netherlands'}]}]
the Netherlands: [{'affiliations': [{'country': 'the Netherlands'}]}]


In [119]:
for i in db.laureates.find(
    filter={'gender':{'$ne':'org'}},
    projection={'bornCountry': 1, 'prizes.affiliations.country': 1},limit=3):
    print("{bornCountry}: {prizes}".format(**doc))

the Netherlands: [{'affiliations': [{'country': 'the Netherlands'}]}]
the Netherlands: [{'affiliations': [{'country': 'the Netherlands'}]}]
the Netherlands: [{'affiliations': [{'country': 'the Netherlands'}]}]


## operator expression

https://www.jianshu.com/p/5f64b0084570

表達式操作符

In [191]:
#$setEquals :Returns true if the input sets have the same distinct elements. Accepts two or more argument expressions.
#$setIsSubset : Returns true if all elements of the first set appear in the second set, including when the first set equals the second set; i.e. not a strict subset. Accepts exactly two argument expressions.
list(
    db.prizes.aggregate([{
        "$project": {
            "allThree": {   #可以直接命名新的name 產生新field
                "$setEquals": ['$laureates.share', ['3']]
            },
            "noneThree": {
                "$not": {
                    "$setIsSubset": [['3'], '$laureates.share']
                }
            }
        }
    }, {
        "$match": {
            "$nor": [{
                "allThree": True
            }, {
                "noneThree": True
            }]
        }
    }]))

[]

In [127]:
pipeline = [{
    '$match': {
        'gender': "org"
    }
}, {
    "$project": {
        "n_prizes": {
            "$size": '$prizes'
        }
    }
}, {
    "$group": {
        "_id": None,   #'_id' :None 表示不指定id 一起加總
        "n_prizes_total": {
            "$sum": '$n_prizes'
        }
    }
}]

print(list(db.laureates.aggregate(pipeline)))

[{'_id': None, 'n_prizes_total': 54}]


In [192]:
#proeject 默認'_id'是被包含的,如果不想被包含可以用'_id':0
from collections import OrderedDict

original_categories = sorted(set(db.prizes.distinct("category", {"year": "1901"})))
pipeline = [
    {"$match": {"category": {"$in": original_categories}}},
    {"$project": {"category": 1, "year": 1}},
    
    # Collect the set of category values for each prize year.
    # addToSet 將category改成set ,改成unique
    {"$group": {"_id": "$year", "categories": {"$addToSet": "$category"}}},
    
    # Project categories *not* awarded (i.e., that are missing this year).
    {"$project": {"missing": {"$setDifference": [original_categories, "$categories"]}}},
    
    # Only include years with missing categories
    {"$match": {"missing.0": {"$exists": True}}},
    
    # Sort in reverse chronological order. Note that "_id" is a distinct year at this stage.
    {"$sort": OrderedDict([("_id", -1)])},
]
for doc in db.prizes.aggregate(pipeline):
    print("{year}: {missing}".format(year=doc["_id"],missing=", ".join(sorted(doc["missing"]))))

2018: literature
1972: peace
1967: peace
1966: peace
1956: peace
1955: peace
1948: peace
1943: literature, peace
1939: peace
1935: literature
1934: physics
1933: chemistry
1932: peace
1931: physics
1928: peace
1925: medicine
1924: chemistry, peace
1923: peace
1921: medicine
1919: chemistry
1918: literature, medicine, peace
1917: chemistry, medicine
1916: chemistry, medicine, peace, physics
1915: medicine, peace
1914: literature, peace


## unwind	indexOfBytes
$unwind : 拆分操作符，用于将数组中的每一个值拆分为单独的文档 

$indexOfBytes: Searches a string for an occurence of a substring and returns the UTF-8 byte index of the first occurence. If the substring is not found, returns -1.

In [136]:
pipeline = [
    {"$project": {"bornCountry": 1, "prizes.affiliations.country": 1}},
  
    # Ensure a single prize affiliation country per pipeline document
    {"$unwind": "$prizes"},
    {"$unwind": "$prizes.affiliations"},
  
    # Filter out "empty" prize-affiliation-country values
    {"$match": {"prizes.affiliations.country": {"$in": db.laureates.distinct("prizes.affiliations.country")}}},
    {"$project": {"affilCountrySameAsBorn": {
        "$gte": [{"$indexOfBytes": ["$prizes.affiliations.country", "$bornCountry"]}, 0]}}},
  
    # Count documents grouped by value (either True or False) for "$affilCountrySameAsBorn"
   {"$group": {"_id": "$affilCountrySameAsBorn", "count": {"$sum": 1}}},
]
for doc in db.laureates.aggregate(pipeline): 
    print(doc)

{'_id': True, 'count': 954}
{'_id': False, 'count': 522}


In [197]:
db.laureates.find_one({})

{'_id': ObjectId('5c7a5903db0b51193a88b4da'),
 'id': '1',
 'firstname': 'Wilhelm Conrad',
 'surname': 'Röntgen',
 'born': '1845-03-27',
 'died': '1923-02-10',
 'bornCountry': 'Prussia (now Germany)',
 'bornCountryCode': 'DE',
 'bornCity': 'Lennep (now Remscheid)',
 'diedCountry': 'Germany',
 'diedCountryCode': 'DE',
 'diedCity': 'Munich',
 'gender': 'male',
 'prizes': [{'year': '1901',
   'category': 'physics',
   'share': '1',
   'motivation': '"in recognition of the extraordinary services he has rendered by the discovery of the remarkable rays subsequently named after him"',
   'affiliations': [{'name': 'Munich University',
     'city': 'Munich',
     'country': 'Germany'}]}]}

In [211]:
for x in db.laureates.aggregate([
    {'$project': {'gender':1,'prizes.year':1,'_id':0}},
    {'$unwind': '$gender'}
]):
    print(x)

{'gender': 'male', 'prizes': [{'year': '1901'}]}
{'gender': 'male', 'prizes': [{'year': '1902'}]}
{'gender': 'male', 'prizes': [{'year': '1902'}]}
{'gender': 'male', 'prizes': [{'year': '1903'}]}
{'gender': 'male', 'prizes': [{'year': '1903'}]}
{'gender': 'female', 'prizes': [{'year': '1903'}, {'year': '1911'}]}
{'gender': 'male', 'prizes': [{'year': '1904'}]}
{'gender': 'male', 'prizes': [{'year': '1905'}]}
{'gender': 'male', 'prizes': [{'year': '1906'}]}
{'gender': 'male', 'prizes': [{'year': '1907'}]}
{'gender': 'male', 'prizes': [{'year': '1908'}]}
{'gender': 'male', 'prizes': [{'year': '1909'}]}
{'gender': 'male', 'prizes': [{'year': '1909'}]}
{'gender': 'male', 'prizes': [{'year': '1910'}]}
{'gender': 'male', 'prizes': [{'year': '1911'}]}
{'gender': 'male', 'prizes': [{'year': '1912'}]}
{'gender': 'male', 'prizes': [{'year': '1913'}]}
{'gender': 'male', 'prizes': [{'year': '1914'}]}
{'gender': 'male', 'prizes': [{'year': '1915'}]}
{'gender': 'male', 'prizes': [{'year': '1915'}]}


## lookup
$lookup : 连接操作符，用于连接同一个数据库中另一个集合，并获取指定的文档，类似于populate

$lookup:
      {
        from: "users",
        localField: "author",
        foreignField: "name",
        as: "author"
      }

In [137]:
pipeline = [
    # Unwind the laureates array
    {"$unwind": "$laureates"},
    {"$lookup": {"from": "laureates", "foreignField": "id",
                 "localField": "laureates.id", "as": "laureate_bios"}},
  
    # Unwind the new laureate_bios array
    {"$unwind": "$laureate_bios"},
    {"$project": {"category": 1, "bornCountry": "$laureate_bios.bornCountry"}},
  
    # Collect bornCountry values associated with each prize category
    {"$group": {"_id": "$category", "bornCountries": {"$addToSet": "$bornCountry"}}},
  
    # Project out the size of each category's (set of) bornCountries
    {"$project": {"category": 1, "nBornCountries": {"$size": "$bornCountries"}}},
    {"$sort": {"nBornCountries": -1}},
]
for doc in db.prizes.aggregate(pipeline): 
    print(doc)

{'_id': 'literature', 'nBornCountries': 55}
{'_id': 'peace', 'nBornCountries': 50}
{'_id': 'chemistry', 'nBornCountries': 48}
{'_id': 'medicine', 'nBornCountries': 44}
{'_id': 'physics', 'nBornCountries': 44}
{'_id': 'economics', 'nBornCountries': 21}


## addFields

'addFields' stage is equivalent to a '$project' stage that explicitly specifies all existing fields in the input documents and adds the new fields 

添加字段，就是在查询的结果再添加一些字段信息，字段的内容自己决定

In [138]:
pipeline = [
    # Limit results to people; project needed fields; unwind prizes
    {"$match": {"gender": {"$ne": "org"}}},
    {"$project": {"bornCountry": 1, "prizes.affiliations.country": 1}},
    {"$unwind": "$prizes"},
  
    # Count prizes with no country-of-birth affiliation
    # $addFields refactor that stage to be a $project stage instead
    {"$addFields": {"bornCountryInAffiliations": {"$in": ["$bornCountry", "$prizes.affiliations.country"]}}},
    {"$match": {"bornCountryInAffiliations": False}},
    {"$count": "awardedElsewhere"},
]

print(list(db.laureates.aggregate(pipeline)))

[{'awardedElsewhere': 956}]


## insert 

In [139]:
pipeline = [
    {"$match": {"gender": {"$ne": "org"}}},
    {"$project": {"bornCountry": 1, "prizes.affiliations.country": 1}},
    {"$unwind": "$prizes"},
    {"$addFields": {"bornCountryInAffiliations": {"$in": ["$bornCountry", "$prizes.affiliations.country"]}}},
    {"$match": {"bornCountryInAffiliations": False}},
    {"$count": "awardedElsewhere"},
]

# Construct the additional filter stage
added_stage = {"$match": {"prizes.affiliations.country": {"$in": db.laureates.distinct("prizes.affiliations.country")}}}

# Insert this stage into the pipeline
# 插在index為3上
pipeline.insert(3, added_stage)
print(list(db.laureates.aggregate(pipeline)))

[{'awardedElsewhere': 504}]
